In [1]:
from pandora_bayesopt.test_functions.lunar_lander import LunarLanderProblem

In [2]:
import torch
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 42
torch.manual_seed(seed)

In [5]:
dim = 12
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [6]:
def objective_function(X):
    return LunarLanderProblem()(X)

In [7]:
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        input_standardize=True
    )

Optimizer.run(
    num_iterations=2, 
    acquisition_function_class="ThompsonSampling"
)

571 232.21358713670193
932 144.6508599521895
Iteration 0, New point: [0.1702627  0.55547708 0.81363521 0.69487412 0.28854893 0.71227129
 0.61415247 0.518101   0.72595463 0.17144252 0.83310051 0.21575163], New value: -107.1279101203503
Best observed value: 8.647587607025399
Current acquisition value: tensor([[230.1904]])
Cumulative cost: 1.0

Iteration 1, New point: [0.53263212 0.29874453 1.         0.75201223 0.00660404 0.36708079
 0.87548325 0.62125025 1.         0.51979455 0.20831864 0.15394034], New value: -123.69016198532091
Best observed value: 8.647587607025399
Current acquisition value: tensor([[220.5942]])
Cumulative cost: 2.0



In [8]:
best_history = Optimizer.get_best_history()
best_history

[8.647587607025399, 8.647587607025399, 8.647587607025399]